In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import numpy as np
from tqdm.notebook import tqdm
from torchvision import datasets,models,transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import gymnasium
import gym
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
dim = 32
learning_rate = 1e-3

In [ ]:
def train(net,n_epoch = 2):
    loss_fn=nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(net.parameters(),lr=learning_rate)
    for epoch in tqdm(range(n_epoch)):

        running_loss = 0.0
        batchiter = iter(train_loader)
        for i,batch in enumerate(tqdm(batchiter)):
            x_batch,y_batch = batch

            optimizer.zero_grad()
            y_pred = net(x_batch)

            loss = loss_fn(y_pred,y_batch)

            loss.backward()
            optimizer.step()
            running_loss+=loss.item()
            if i % 500 == 499:
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 500))
              running_loss = 0.0
    print("обучение закончено")
    return net

In [ ]:
class Flatten(nn.Module):
    def forward(self,batch):
        return batch.view(batch.size(0),-1)
    
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.cv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.cv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = Flatten()
        self.fc1 = nn.Linear(64 * dim * dim, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = F.relu(self.cv1(x))
        x = F.relu(self.cv2(x))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x


In [ ]:


class DataSelectionEnv(gym.Env):
    def __init__(self):
        super(DataSelectionEnv, self).__init__()
        #загрузили наборы данных
        self.train_data = datasets.CIFAR10("./root", train=True,download=True,transform=transforms.ToTensor())
        self.test_data = datasets.CIFAR10("./root",train=False,download=True,transform=transforms.ToTensor())
        #двумерный массив с индексами изображений по классам
        self.indexes = self.class_select()
        self.num_class = 10
        model = SimpleCNN()
        optimizer = torch.optim.AdamW(self.model.parameters(),lr=1e-3)#нужно создать модель и засунуть сюда это оптимайзер для самой светочной модели
        self.entropy = nn.CrossEntropyLoss()
        total_num_img = 5000

    def sample(self,action):
        index = []
        for i in range(self.num_class):
            num_img = int(action[i]*self.total_num_img)#на основе распределения вероятностей из action рассчитали количество изобьражений в каждом классе
            indexes = np.random.choice(self.class_select[i],num_img,replace=True)#рандомно взяли индексы из списка индексов по классам в нужном количестве 
            index.extend(indexes)
        return Subset(self.train_data,index)#взяли подмножество из train_data по индексам изображений
    
    def class_select(self):
        ls = {i: [] for i in self.num_class}
        for x, (_,label) in enumerate(self.train_data):
            ls[label].append(x)
        return ls
    
    def step(self,action):
        subset = self.sample(action)
        train_dataloader = DataLoader(subset,batch_size=32,shuffle=True)

        self.model.train()
        
    def reset(self):




In [ ]:
class Actor(nn.Module):
    def __init__(self, input_size, num_actions):
        super(Actor,self).__init__()
        self.fc1 = nn.Linear(input_size,64)
        self.fc2 = nn.Linear(64,num_actions)
    def forwars(self,x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=-1)
        return x

In [ ]:
class Critic(nn.Module):
    def __init__(self, input_size):
        super(Critic,self).__init__()
        self.fc1 = nn.Linear(input_size,64)
        self.fc2 = nn.Linear(61,1)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=-1)
        return x

In [ ]:
def actor_critic(actor, critic, episodes, max_steps=2000, gamma=0.99, lr_actor=1e-3, lr_critic=1e-3):
    optimizer_actor = optim.AdamW(actor.parameters(), lr=lr_actor)
    optimizer_critic = optim.AdamW(critic.parameters(), lr=lr_critic)
    stats = {'Actor Loss': [], 'Critic Loss': [], 'Returns': []}

    env = gym.make('CartPole-v1')
    input_size = env.observation_space.shape[0]
    num_actions = env.action_space.n

    for episode in range(1, episodes + 1):
        state = env.reset()[0]
        ep_return = 0
        done = False
        step_count = 0

        while not done and step_count < max_steps:
            state_tensor = torch.FloatTensor(state)
            
            # Actor selects action
            action_probs = actor(state_tensor)
            dist = Categorical(action_probs)
            action = dist.sample()
            
            # Take action and observe next state and reward
            next_state, reward, done, _,_ = env.step(action.item())
            
            # Critic estimates value function
            value = critic(state_tensor)
            next_value = critic(torch.FloatTensor(next_state))
            
            # Calculate TD target and Advantage
            td_target = reward + gamma * next_value * (1 - done)
            advantage = td_target - value
            
            # Critic update with MSE loss
            critic_loss = F.mse_loss(value, td_target.detach())
            optimizer_critic.zero_grad()
            critic_loss.backward()
            optimizer_critic.step()
            
            # Actor update
            log_prob = dist.log_prob(action)
            actor_loss = -log_prob * advantage.detach()
            optimizer_actor.zero_grad()
            actor_loss.backward()
            optimizer_actor.step()
            
            # Update state, episode return, and step count
            state = next_state
            ep_return += reward
            step_count += 1

        # Record statistics
        stats['Actor Loss'].append(actor_loss.item())
        stats['Critic Loss'].append(critic_loss.item())
        stats['Returns'].append(ep_return)

        # Print episode statistics
        print(f"Episode {episode}: Actor Loss: {actor_loss.item():.4f}, Critic Loss: {critic_loss.item():.4f}, Return: {ep_return}, Steps: {step_count}")

    env.close()
    return stats